### **This leads_unpack script is script 1A and 1B for the LEADS data. This will unpack the data, create an mgz file and a scaninfo file with run information for the MPRAGE. It also renames the subject folders to session folders. Then it creates two log files: unpack.log contains all sessions that were unpacked (i.e. run by this script), and batch.recon.list which is erased at the onset of running this script to create a batch list that has not been previously run. So, after running this script, run the batch script batch.recon.sh which calls the batch.recon.list to run the initial recon. When the batch jobs have finished on launchpad, you must then move on to script 1B (see below) before completing the manual edits (working on automating this).**

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

import re
import glob
import os
import pandas as pd
from os import system
import csv
from nipype.interfaces.freesurfer import MRIConvert
import subprocess
import getpass
from pathlib import Path
import shutil
#import nbpackage.mynotebook

dicomdir = "/cluster/animal/scan_data/leads/LEADS/"
unpacklog = "/autofs/cluster/animal/scan_data/leads/recon/unpack.log"
recondir = "/autofs/cluster/animal/scan_data/leads/recon/"
folders = [x for x in os.listdir(dicomdir) if not x.startswith(".")]

# wipe clean batch.recon.list ----------------- either end of file or here?
open(recondir+'batch.recon.list', 'w').close()
pswd= ''

Elements = {'scan_notes': [''],'loni_overallpass': [''], 'download_date':[''], 'xnat_upload':[''], 'recon_path':[''],'recon_notes':[''], 'dickerson_overallpass':['']}
df = pd.DataFrame(Elements, columns= ['scan_notes', 'loni_overallpass', 'download_date','xnat_upload','recon_path','recon_notes','dickerson_overallpass'])

In [3]:
def passprompt():
    USER= input("Please enter your USERNAME for launchpad acess: ")
    print('PASSWORD: ')
    PASS= getpass.getpass()
    return USER, PASS

In [4]:
[USER, PASS] =passprompt()

Please enter your USERNAME for launchpad acess: rje11
PASSWORD: 
········


In [5]:
def parse_scaninfo(origpath):
    with open(origpath+'/scan.info', 'r') as in_file:
        for line in in_file:
            editline = line.split()
            with open(origpath+'/scaninfo.csv', 'w') as result:
                wr = csv.writer(result, dialect='excel')
                wr.writerow(editline)
            result.close()
    in_file.close()
    
def shortversion(long):
    size = len(long)
    x = 0
    while x ==0:
        if (long[-1] == '0') or (long[-1] == '.'):
            long = long[:-1]
        else:
            x =1
    return long



In [9]:
for substring in folders:
    subjectdir = recondir+substring
    #if substring not in open(unpacklog).read():
    if "_" not in substring:
        print("will be unpacking "+ substring)

In [14]:
%%time 
# APPROX Wall time: 46.5 s PER SUBJECT

# load in new version of excel doc from LONI with exclusion criteria (add comments to ours somehow)

folders = ['LDS0370007']

for substring in folders:
    subjectdir = recondir+substring
    #if substring not in open(unpacklog).read():
    if "a" == "a":
    #if "_" not in substring:
        print('unpacking ' + substring + " ----------------------- ")
        os.makedirs(recondir+substring+'/mri/orig/')

        if substring == 'LDS0370006':
            tmp = '/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370006/Accelerated_Sagittal_MPRAGE/2018-07-26_10_41_42.0/S708999/'
            MPRAGE_list = glob.glob(tmp, recursive=True)
        elif substring == 'LDS0370007':
            tmp = '/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370007/Accelerated_Sagittal_MPRAGE/2018-08-01_10_49_41.0/S711584/'
            MPRAGE_list = glob.glob(tmp, recursive=True)         
        elif substring == 'LDS0370013':
            tmp = '/autofs/cluster/animal/scan_data/leads/LEADS/LDS0370013/Accelerated_Sagittal_MPRAGE/2018-08-22_08_55_38.0/S722833/'
            MPRAGE_list = glob.glob(tmp, recursive=True)
        else:
            # check number of date folders; check number of session folders
            MPRAGE_list = glob.glob(dicomdir+substring + '/Accelerated_Sagittal_MPRAGE/*/*/', recursive=True)
        #elif len(MPRAGE_list) > 1:
        #    print("There are multiple MPRAGEs for "+substring+". Unpacking only the first; check others.")
    
        # for now, proceed with unpacking only the first valid run (given multiple)
        # check for errors and record RUN#
        cmdstring = 'unpacksdcmdir -src %s -targ %s -scanonly %s/scan.info' % (MPRAGE_list[0], subjectdir, subjectdir)
        system(cmdstring)

        parse_scaninfo(subjectdir)
        scan_info = pd.read_csv(subjectdir+'/scaninfo.csv', header=None)
        check = scan_info.iloc[0,2] # first row (only row); second col (validity)
        
        #convert from dicom's MPRAGE to mgz (allow for more than one)
        if check != 'ok':
            print(MPRAGE_list[0]+" IS AN INVALID RUN (I.E. NEEDS TO BE OMMITTED FROM ANALYSIS). SKIPPING THE UNPACKING.")
        else:
            # make list of all files within MPRAGE_list[0]
            dicomlist = os.listdir(MPRAGE_list[0])
            with open(subjectdir+'/dicomlist.txt', 'w') as f:
                for item in dicomlist:
                    f.write("%s\n" % (item))
                    
            mc = MRIConvert()
            mc.inputs.in_file = MPRAGE_list[0]+os.listdir(MPRAGE_list[0])[0] # first file
            #mc.inputs.sdcm_list = subjectdir+'/dicomlist.txt' # no need for this
            mc.inputs.out_file = subjectdir+'/mri/orig/001.mgz'
            mc.inputs.out_type = 'mgz'
            system(mc.cmdline)

            #obtain date: must change folder names in both leadsir/recon and /leadsdir/LEADS
            datestring = scan_info.iloc[0,7] 
            date = re.search('raw_'+r'+[0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]', datestring).group()[4:]

            #rename both folders
            os.rename(recondir+substring, recondir+substring+'_'+date)
            os.rename(dicomdir+substring, dicomdir+substring+'_'+date) 
            
            # create batch.recon.list to launch recon list
            with open(recondir+'batch.recon.list', "a") as myfile:
                myfile.write(substring+'_'+date+'\n')

            ##add substring to log
            with open(unpacklog, "a") as myfile:
                myfile.write(substring+'_'+date+'\n') 
            
            # save an empty dataframe for scannotes (will add info after recon)
            df.to_csv(subjectdir+'_'+date+'/scannotes.csv')



unpacking LDS0370007 ----------------------- 
CPU times: user 27.1 ms, sys: 26.5 ms, total: 53.6 ms
Wall time: 47.5 s


In [155]:
# WORKS. This will not re-run recons that have been processed.
#I need to somehow to connect script 2 to this -- if I can make the script wait until
# the jobs are done.
from paramiko import SSHClient
host="launchpad"
user=USER
pw=PASS
client=SSHClient()
client.load_system_host_keys()
client.connect(host,username=user,password=pw, look_for_keys=False)
stdin, stdout, stderr = client.exec_command('(cd /autofs/cluster/animal/scan_data/leads/analyses; ./batch.recon.sh)')
print("stderr: ", stderr.readlines())
print("pwd: ", stdout.readlines())

stderr:  ['Opening pbsjob_321\n']
pwd:  ['Beginning batch submission..\n', 'qsub -V -S /bin/sh  -m abe -M rje11   -l nodes=1:ppn=1,vmem=7gb -r n   /pbs/rje11/pbsjob_321 \n', '17040077.launchpad.nmr.mgh.harvard.edu\n', 'SUBMITTED JOB for LDS0370007_20180801\n']


In [257]:
# --------------------------------------------------------------------------------------------------- #
# check the status somehow: do every hour? or set a timer when I start?
import time

# can I run several jobs through the function simultaneously?

#for now only works for one job
runningjobs = [] # qeued or running
staticjobs = [] # completed with exit status 0 or 1
pause= 100 #14400 #4 hours in seconds

while submission != []:
    stdin, stdout, stderr = client.exec_command('qstat -u rje11')
    submission = "pwd: ", stdout.readlines()
    n=5 #start of jobs print
    try:
        sublist = (submission[1])[n]
        job=sublist.split()
        username = job[1]
        jobname = job[3] # job name col
        jobstatus=job[9]
        if jobstatus=="C": #change back to C
            print(jobname+" status: complete")
            staticjobs.append(jobname)
            # check if completed with no errors
            stdin2, stdout2, stderr2 = client.exec_command('tail -n 1 /pbs/'+username+'/'+jobname+'.status')
            complete_status = "pwd: ", stdout2.readlines()
            exit = (complete_status[1][0][-2])
            subj = (complete_status[1][0])
            # add the session_name 
            stdin3, stdout3, stderr3 = client.exec_command('head -n 4 /pbs/'+username+'/'+jobname+'.status')
            session_name = "pwd: ", stdout3.readlines()
            subj = (session_name[1])[3] # take name out of this string
            if exit == '1':
                print(jobname+' exited with status 1. CHECK THIS SESSION and RE-RUN.')
            elif exit == '0':
                print("Will continue with execution")
            # make other script a function and call it from here.
            # need the directory name
        else:
            runningjobs.append(jobname)
            print(jobname+" status: "+jobstatus)
        n=n+1
    except(IndexError):
        pass
    time.sleep(pause) # is this bad practice? It runs until completely done with job..



pbsjob_321 status: R


KeyboardInterrupt: 

### **This is script 1B, which should be run after script 1A. This relabels and relocates the original recon output folder to a subdirectory within the sesion folder. The subdirectory is named according to FSversion_Run#. This script also makes a copy of this folder and renames it "unedit.FSversion_Run#'. The edit folder is the one that will be manually edited and while making edits, you can insert comments into the scannotes.csv in the session directory. NOTE: the unedit.recon output is symlinked to the analysis folder. After re-running the manually edited recons, you can rename the folder to edit.recon so that it is known which have been completed. After manual edit recon jobs are finished running, you can move on to LEADS_redcapfill (scripts 2A and 2B) to create the data matrix which will be used to import the data into RedCap)**

In [ ]:
# IN ORDER TO RUN THIS-- I NEED TO WAIT FOR THE RECONS TO FINISH-- how can I automate this?

# this makes a copy of all initial recopn output and labels them according to FSV_run#
# then creates symlinks to an analysis folder

realrecons= '/autofs/cluster/animal/scan_data/leads/recon/'
analysesdir = '/autofs/cluster/animal/scan_data/leads/analyses/'

folders = [x for x in os.listdir(realrecons) if x.startswith("LDS")]

#folders = ['LDS0370020_20181212', 'LDS0110021_20181016']
freesurfer_dirs = ['mri', 'stats', 'tmp', 'trash', 'touch', 'label', 'surf', 'scripts']

for sub in folders:
    print(sub)
    
    try:
        # obtain FS version
        versionfile = open(realrecons+sub+'/scripts/build-stamp.txt', 'r')
        versionstring = versionfile.read()
        version = versionstring.split('-')
        result = [i for i in version if i.startswith('v')][0]
        long = result[1:]
        vlabel = 'FS'+shortversion(long)

        # obtain run number
        with open(realrecons+sub+'/scaninfo.csv','r') as f:
            reader = csv.reader(f)
            scan_list = list(reader)
            runstring = scan_list[0][0] # run
            if len(runstring) == 1:
                runstring = '0'+runstring
        recon_name = vlabel+'_'+runstring

        print(recon_name)
    except(FileNotFoundError):
        print(sub+" freesurfer files not in directory.")
    
    # now go into each subject folder and create a folder with recon_name
    try:
        os.mkdir(realrecons+sub+'/'+recon_name)
    except(FileExistsError):
        print(sub+" original recon folder already created.")
    
    # move all subfolders into this recon_name folder
    for fsdir in freesurfer_dirs:
        FS = Path(realrecons+sub+'/'+fsdir)
        if FS.exists():
            shutil.move(realrecons+sub+'/'+fsdir, realrecons+sub+'/'+recon_name+'/'+fsdir)
            
    try:
        recon_name2 = 'unedit.'+recon_name
        FS2 = Path(realrecons+sub+'/'+recon_name2)
        shutil.copytree(realrecons+sub+'/'+recon_name, FS2) # if edit.recon folder does not already exist, add to log
        with open(realrecons+'to_edit.list\n', "a") as myfile:
            myfile.write(sub)
    except(FileExistsError):
        print(sub+" unedit.recon folder already created.")
        
    #Now create symlink to edit. in analysis folder!!
    try:
        os.symlink(realrecons+sub+'/'+recon_name2, analysesdir+sub)
    except(FileExistsError):
        print(sub+" in analyses is already linked to unedit.recon.")